In [1]:
import polars as pl

In [2]:
#1. Les inn kjoretoyinfo_fra_[putt_inn_år].parquet. Bruk pl.scan_parquet.
df=pl.scan_parquet("kjoretoyinfo_fra_2000.parquet")

In [3]:
#2. “tekn_reg_f_g_n” er dato for førstegangsregistrering i Norge. Konverter denne kolonnen til en datetime. Gjør tilsvarende for “neste_pkk” og “tekn_reg_eier_dato”.
df = df.with_columns([
    pl.col("tekn_reg_f_g_n").cast(pl.Utf8).str.to_date(format="%Y%m%d").alias("tekn_reg_f_g_n")
])

In [4]:
df_t = df.collect()

In [5]:
df = df.with_columns([
    pl.col("tekn_neste_pkk").str.to_date(format="%Y%m%d", strict=False).alias("tekn_neste_pkk")
])


In [6]:
df_t = df.collect()

In [7]:
df = df.with_columns([
    pl.col("tekn_reg_eier_dato").cast(pl.Utf8).str.to_date(format="%Y%m%d").alias("tekn_reg_eier_dato"),
])

In [8]:
df_t = df.collect()
df_t

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_kjtgrp,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt
i64,date,date,i64,i64,str,str,str,str,date,i64,i64
20211013,2021-10-13,2021-10-13,3900,315,"""VITO""","""05""","""2""",null,2025-10-13,2,3
20200129,2020-01-29,2020-01-29,5750,315,"""CADDY""","""06""","""2""","""20231205""",2026-01-29,2,2
20231129,2023-11-29,2023-11-29,2600,101,"""Mustang Mach-E…","""06""","""5""",null,2027-11-29,1,5
20200127,2020-01-27,2020-01-27,5750,315,"""CADDY""","""01""","""2""",null,2024-01-27,2,2
20180731,2019-01-30,2019-01-30,2600,315,"""F-150 Platinum…","""06""","""2""","""20220503""",2024-05-03,2,2
…,…,…,…,…,…,…,…,…,…,…,…
20031127,2003-11-27,2017-06-10,4350,101,"""ZAFIRA-A""","""11""","""1""","""20190514""",2021-04-30,0,7
20140117,2014-01-17,2018-07-31,2600,315,"""RANGER""","""05""","""2""","""20220511""",2024-06-30,2,2
20080331,2008-03-31,2023-07-28,3930,101,"""MX-5""","""05""","""1""","""20230816""",2025-08-16,0,2


In [9]:
print(df_t["tekn_farge"].head(3))

shape: (3,)
Series: 'tekn_farge' [str]
[
	"05"
	"06"
	"06"
]


In [10]:
#3. Join inn navnet på fargen, det vil si, join “tekn_farge” med “kode” i fargekode.csv. Pass på at du ikke mister noen biler som vi ikke vet fargen på.
df_fargekode = pl.read_csv("fargekode.csv", separator=";", dtypes={"kode":str}).rename({"kode": "tekn_farge"})

In [11]:
df_fargekode

tekn_farge,beskrivelse
str,str
"""1""","""Hvit (også ant…"
"""2""","""Rød (også burg…"
"""3""","""Blå"""
"""4""","""Grønn (også gr…"
"""5""","""Grå"""
…,…
"""12""","""Gull (også bro…"
"""13""","""Rosa"""
"""14""","""Fiolett (også …"


In [12]:
df_joined = df_t.select(pl.col("tekn_farge")).join(
    df_fargekode.select(pl.col("tekn_farge"), pl.col("beskrivelse")),
    on="tekn_farge",
    how="left"
)
df_joined

tekn_farge,beskrivelse
str,str
"""05""",null
"""06""",null
"""06""",null
"""01""",null
"""06""",null
…,…
"""11""","""Sølv (også søl…"
"""05""",null
"""05""",null


In [13]:
print(df_t.head(3))

shape: (3, 12)
┌──────────────┬────────────────┬────────────────────┬────────────┬───┬────────────────┬────────────────┬───────────────────┬──────────────────────────┐
│ tekn_reg_f_g ┆ tekn_reg_f_g_n ┆ tekn_reg_eier_dato ┆ tekn_merke ┆ … ┆ tekn_siste_pkk ┆ tekn_neste_pkk ┆ tekn_aksler_drift ┆ tekn_sitteplasser_totalt │
│ ---          ┆ ---            ┆ ---                ┆ ---        ┆   ┆ ---            ┆ ---            ┆ ---               ┆ ---                      │
│ i64          ┆ date           ┆ date               ┆ i64        ┆   ┆ str            ┆ date           ┆ i64               ┆ i64                      │
╞══════════════╪════════════════╪════════════════════╪════════════╪═══╪════════════════╪════════════════╪═══════════════════╪══════════════════════════╡
│ 20211013     ┆ 2021-10-13     ┆ 2021-10-13         ┆ 3900       ┆ … ┆ null           ┆ 2025-10-13     ┆ 2                 ┆ 3                        │
│ 20200129     ┆ 2020-01-29     ┆ 2020-01-29         ┆ 5750       ┆

In [14]:
print(df_fargekode.columns)

['tekn_farge', 'beskrivelse']


In [15]:
#4. Lag en kolonne “elbil” som er sann hvis drivstofftypen er kun elektrisk, false ellers.
df_t = df_t.with_columns(
    er_elektrisk=pl.col("tekn_drivstoff").eq(pl.lit("5"))
)


df_t

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_kjtgrp,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt,er_elektrisk
i64,date,date,i64,i64,str,str,str,str,date,i64,i64,bool
20211013,2021-10-13,2021-10-13,3900,315,"""VITO""","""05""","""2""",null,2025-10-13,2,3,false
20200129,2020-01-29,2020-01-29,5750,315,"""CADDY""","""06""","""2""","""20231205""",2026-01-29,2,2,false
20231129,2023-11-29,2023-11-29,2600,101,"""Mustang Mach-E…","""06""","""5""",null,2027-11-29,1,5,true
20200127,2020-01-27,2020-01-27,5750,315,"""CADDY""","""01""","""2""",null,2024-01-27,2,2,false
20180731,2019-01-30,2019-01-30,2600,315,"""F-150 Platinum…","""06""","""2""","""20220503""",2024-05-03,2,2,false
…,…,…,…,…,…,…,…,…,…,…,…,…
20031127,2003-11-27,2017-06-10,4350,101,"""ZAFIRA-A""","""11""","""1""","""20190514""",2021-04-30,0,7,false
20140117,2014-01-17,2018-07-31,2600,315,"""RANGER""","""05""","""2""","""20220511""",2024-06-30,2,2,false
20080331,2008-03-31,2023-07-28,3930,101,"""MX-5""","""05""","""1""","""20230816""",2025-08-16,0,2,false


In [16]:
#5.Hent ut bare kolonnene:
selected_columns = [
    "tekn_reg_f_g_n",
    "tekn_reg_eier_dato",
    "tekn_aksler_drift",
    "tekn_merke",
    "tekn_modell",
    "tekn_drivstoff",
    "tekn_neste_pkk",
    "er_elektrisk",
    "tekn_farge",
    "tekn_kjtgrp"
]

df_selected = df_t.select(selected_columns)

In [17]:
#6. Skriv hele datasettet til en fil: kjoretoyinfo_preppet.parquet
df_p=df_selected.write_parquet('kjoretoyinfo_preppet.parquet')

In [18]:
df_preppet = pl.read_parquet('kjoretoyinfo_preppet.parquet')

print(df_preppet.head())

shape: (5, 10)
┌────────────────┬────────────────────┬───────────────────┬────────────┬───┬────────────────┬──────────────┬────────────┬─────────────┐
│ tekn_reg_f_g_n ┆ tekn_reg_eier_dato ┆ tekn_aksler_drift ┆ tekn_merke ┆ … ┆ tekn_neste_pkk ┆ er_elektrisk ┆ tekn_farge ┆ tekn_kjtgrp │
│ ---            ┆ ---                ┆ ---               ┆ ---        ┆   ┆ ---            ┆ ---          ┆ ---        ┆ ---         │
│ date           ┆ date               ┆ i64               ┆ i64        ┆   ┆ date           ┆ bool         ┆ str        ┆ i64         │
╞════════════════╪════════════════════╪═══════════════════╪════════════╪═══╪════════════════╪══════════════╪════════════╪═════════════╡
│ 2021-10-13     ┆ 2021-10-13         ┆ 2                 ┆ 3900       ┆ … ┆ 2025-10-13     ┆ false        ┆ 05         ┆ 315         │
│ 2020-01-29     ┆ 2020-01-29         ┆ 2                 ┆ 5750       ┆ … ┆ 2026-01-29     ┆ false        ┆ 06         ┆ 315         │
│ 2023-11-29     ┆ 2023-11-29    